## Solve an inverse tarffic problem over polynomials of degree at most d

## Optionally use a regularizer from the poly kernel

In [1]:
%run ../Python_files/util.py

No dicts found; please check load_dicts...


In [2]:
import json

In [3]:
# read in node-link incidence matrtix
with open('node_link_incidence_Sioux.json', 'r') as json_file:
    N_dict = json.load(json_file)
    
N = zload('node_link_incidence_Sioux.pkz')

In [4]:
N.shape

(24, 76)

In [5]:
# define polynomial kernel with degree d
def kernel(x, y, c, d):
    return (c + x*y) ** d

In [6]:
capac_list = []
free_flow_time_list = []
capac_dict = {}
free_flow_time_dict = {}

with open('SiouxFalls_net.txt', 'r') as f:
    read_data = f.readlines()

for row in read_data:
    if len(row.split()) == 11:
        key = row.split()[0] + ',' + row.split()[1]
        capac_list.append(float(row.split()[2]))
        free_flow_time_list.append(float(row.split()[3]))
        capac_dict[key] = float(row.split()[2])
        free_flow_time_dict[key] = float(row.split()[3])

In [7]:
numNode = N.shape[0]
numLink = N.shape[1]
assert(numLink == len(capac_list))

In [8]:
flow_list = []
flow_dict = {}

with open('SiouxFallsFlow.txt', 'r') as f:
    read_data = f.readlines()

for row in read_data:
    if len(row.split()) == 4:
        key = row.split()[0] + ',' + row.split()[1]
        flow_list.append(float(row.split()[2]))
        flow_dict[key] = float(row.split()[2])
#         print(row.split())

In [9]:
flow_normalized = [flow_list[i]/capac_list[i] for i in range(numLink)]

In [10]:
len(flow_normalized)

76

In [11]:
# construct kernel matrix; take d=5 for instance
c = 1.0
d = 4

phi = np.zeros((numLink, numLink))
for i in range(numLink):
    for j in range(numLink):
        phi[i, j] = kernel(flow_normalized[i], flow_normalized[j], c, d)

In [12]:
phi.shape

(76, 76)

In [13]:
np.linalg.matrix_rank(phi)

5

In [14]:
# read in demand data
with open('demands_Sioux.json', 'r') as json_file:
    demands = json.load(json_file)

In [15]:
od_list = []
for i in range(numNode + 1)[1:]:
    for j in range(numNode + 1)[1:]:
        if i != j:
            key = '(' + str(i) + ',' + str(j) + ')'
            od_list.append(key)

In [16]:
N.shape

(24, 76)

In [17]:
N_t = np.transpose(N)

In [18]:
N_t.shape

(76, 24)

In [19]:
model = Model("InverseVI_Sioux")

alpha = {}
for i in range(numLink):
    key = str(i)
    alpha[key] = model.addVar(name='alpha_' + key)
    
epsilon = model.addVar(name='epsilon')

yw = {}
for od in od_list:
    for i in range(numNode):
        key = od + str(i)
        yw[key] = model.addVar(name='yw_' + key)
        
model.update()

In [20]:
N.shape

(24, 76)

In [21]:
# add dual feasibility constraints
for od in od_list:
    for a in range(numLink):
        model.addConstr(sum([N[i, a] * yw[od+str(i)] for i in range(numNode)]) <= 
                        free_flow_time_list[a] * sum([alpha[str(j)] * phi[j, a] for j in range(numLink)]))
        
model.update()

In [22]:
# add increasing constraints
myList = flow_normalized
flow_sorted_idx = sorted(range(len(myList)),key=lambda x:myList[x])
for i in range(numLink):
    if (i < numLink-1):
        a_i_1 = flow_sorted_idx[i]
        a_i_2 = flow_sorted_idx[i+1]
        model.addConstr(sum([alpha[str(j)] * phi[j, a_i_1] for j in range(numLink)]) <= 
                        sum([alpha[str(j)] * phi[j, a_i_2] for j in range(numLink)]))
model.update()

In [23]:
model.addConstr(epsilon >= 0)

model.update()

In [24]:
# str(int(od.split(',')[0].split('(')[1])-1), str(int(od.split(',')[1].split(')')[0])-1)

In [25]:
# add primal-dual gap constraint
primal_cost = sum([free_flow_time_list[a] * sum([alpha[str(j)] * phi[j, a] for j in range(numLink)]) 
                   for a in range(numLink)])
dual_cost = sum([demands[od] * (yw[od + str(int(od.split(',')[1].split(')')[0])-1)] - 
                                yw[od + str(int(od.split(',')[0].split('(')[1])-1)]) 
                 for od in od_list])

model.addConstr(primal_cost - dual_cost <= epsilon)
model.addConstr(dual_cost - primal_cost <= epsilon)
        
model.update()

In [26]:
# add normalization constraint
model.addConstr(sum([alpha[str(j)] * phi[j, 0] for j in range(numLink)]) == 1)

model.update()

In [27]:
# Set objective
obj = 0

gama = 1e4

for i in range(numLink):
    for j in range(numLink):
        obj += alpha[str(i)] * phi[i, j] * alpha[str(j)]

obj += gama * epsilon

model.setObjective(obj)

In [28]:
model.optimize()

Optimize a model with 42031 rows, 13325 columns and 3280147 nonzeros
Model has 2926 quadratic objective terms
Coefficient statistics:
  Matrix range    [7e-04, 2e+05]
  Objective range [1e+04, 1e+04]
  Bounds range    [0e+00, 0e+00]
  RHS range       [1e+00, 1e+00]
Presolve removed 76 rows and 0 columns
Presolve time: 1.31s
Presolved: 41955 rows, 13325 columns, 3274598 nonzeros
Presolved model has 2926 quadratic objective terms
Ordering time: 0.04s

Barrier statistics:
 Dense cols : 76
 Free vars  : 6
 AA' NZ     : 3.420e+06
 Factor NZ  : 4.284e+06 (roughly 60 MBytes of memory)
 Factor Ops : 4.430e+08 (less than 1 second per iteration)
 Threads    : 12

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   2.62144000e+12 -8.87730854e-01  5.27e+04 1.00e+00  6.18e+07     2s
   1   1.81341176e+12 -1.74458094e+09  3.64e+04 3.52e+03  4.56e+07     3s
   2   1.05135923e+12 -4.58242930e+09  2.10e+04 2.67e+03  2.68e+

In [29]:
alpha

{'0': <gurobi.Var alpha_0 (value 0.887531610538)>,
 '1': <gurobi.Var alpha_1 (value 1.92160379967e-05)>,
 '10': <gurobi.Var alpha_10 (value 2.80808029504e-08)>,
 '11': <gurobi.Var alpha_11 (value 4.2279596946e-09)>,
 '12': <gurobi.Var alpha_12 (value 8.58760485512e-09)>,
 '13': <gurobi.Var alpha_13 (value 9.0098482726e-08)>,
 '14': <gurobi.Var alpha_14 (value 4.21942621901e-09)>,
 '15': <gurobi.Var alpha_15 (value 4.10537767517e-10)>,
 '16': <gurobi.Var alpha_16 (value 1.8512196381e-08)>,
 '17': <gurobi.Var alpha_17 (value 7.58262385812e-06)>,
 '18': <gurobi.Var alpha_18 (value 1.19377752819e-09)>,
 '19': <gurobi.Var alpha_19 (value 2.33173696793e-08)>,
 '2': <gurobi.Var alpha_2 (value 0.00048113245235)>,
 '20': <gurobi.Var alpha_20 (value 4.53253698177e-07)>,
 '21': <gurobi.Var alpha_21 (value 5.42532515419e-09)>,
 '22': <gurobi.Var alpha_22 (value 8.39135387952e-09)>,
 '23': <gurobi.Var alpha_23 (value 6.42166900269e-07)>,
 '24': <gurobi.Var alpha_24 (value 1.13690760212e-08)>,
 '25'